In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls
!pwd
!cd /content/drive
print('done')

Mounted at /content/drive
drive  sample_data
/content
done


In [ ]:
ls "/content/drive/My Drive/UIA/SnowBall"

ls: cannot access '/content/drive/My Drive/UIA/SnowBall': No such file or directory


In [ ]:
cd "/content/drive/My Drive/UIA/Snowball"

/content/drive/My Drive/UIA/Snowball


In [ ]:
import sys
import requests
import json

In [ ]:
import pandas as pd
similar=pd.read_csv("/content/drive/My Drive/UIA/Snowball/SimilarUIAELayer2.csv",index_col=0)
similar.columns

Index(['appID'], dtype='object')

In [ ]:
similar.shape

(2553, 1)

In [ ]:
similar.reset_index(drop=True,inplace=True)

In [ ]:
import sys

In [ ]:
import sqlite3
import pandas as pd
# Create your connection.
cnx = sqlite3.connect('SnowBallTill15kLayer2.db')

df = pd.read_sql_query("SELECT * FROM reviews", cnx)

In [ ]:
pd.read_sql_query("SELECT * FROM Apps", cnx)

,appID,appName,genre,url,baseid,baseapp
0,1188441283,1188441283,1188441283,1188441283,1188441283,1188441283
1,384905964,384905964,384905964,384905964,384905964,384905964
2,990577711,990577711,990577711,990577711,990577711,990577711
3,1296783740,1296783740,1296783740,1296783740,1296783740,1296783740
4,1018154760,1018154760,1018154760,1018154760,1018154760,1018154760
...,...,...,...,...,...,...
547,894045208,894045208,894045208,894045208,894045208,894045208
548,933016168,933016168,933016168,933016168,933016168,933016168
549,734328438,734328438,734328438,734328438,734328438,734328438
550,752005174,752005174,752005174,752005174,752005174,752005174


In [ ]:
len(df['appID'].unique())

3462

In [ ]:
filepath = '/content/drive/My Drive/UIA/Snowball/SimilarUIAELayer2.csv'

if sys.version_info >= (3,):
    f = open(filepath, 'r', encoding='utf-8')
    import urllib.parse as ul
else:
    f = open(filepath, 'r')
    import urllib as ul
    
import csv

app_list = []
app_list_obj = []
header = []

with f as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',',quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for tp in csvreader:
        tp=tp[1:]
        print(tp)
        if tp[0] == 'appID':
            print("hi")
            header = tp
            continue
        else:
            app_list.append(tp)
            app = {}
            for hid in range(len(header)):
                app[header[hid]] = tp[hid].encode().decode('utf-8', 'ignore')
            app_list_obj.append(app)
f.close()            
sys.stdout.write('# Apps: {}\n'.format(len(app_list)))

In [ ]:
user_agent = 'iTunes/12.6 (Windows; Microsoft Windows 7 Business Edition Service Pack 1 (Build 7601)) AppleWebKit/536.27.1'
store_front = '143441-1,32'

headers = {
    'User-Agent': user_agent,
    'X-Apple-Store-Front': store_front
}

review_step = 1000
number_of_trying = 10

In [ ]:
pwd

'/content/drive/My Drive/UIA/Snowball'

In [ ]:
import sqlite3
conn = sqlite3.connect('SnowBallTill15kLayer2.db')
curr = conn.cursor()
curr.execute('DROP TABLE IF EXISTS Apps')
curr.execute('DROP TABLE IF EXISTS Reviews')
curr.execute('CREATE TABLE "Apps" ( ' \
	'"appID"	INTEGER,' \
	'"appName"	TEXT,' \
	'"genre"	TEXT,' \
	'"url"	TEXT,' \
	'"baseid"	INTEGER,' \
	'"baseapp"	TEXT' \
	# 'PRIMARY KEY("appID")' \
')')

curr.execute('CREATE TABLE "Reviews" (' \
    '"recordID"	INTEGER PRIMARY KEY AUTOINCREMENT,' \
	'"reviewID"	INTEGER,' \
	'"appID"	INTEGER,' \
    '"timestamp"	TEXT,' \
	'"rating"	INTEGER,' \
	'"name"	TEXT,' \
	'"title"	TEXT,' \
	'"body"	TEXT' \
')')

conn.commit()


In [ ]:
df = pd.read_sql_query("SELECT * FROM Reviews", conn)

In [ ]:
df

,recordID,reviewID,appID,timestamp,rating,name,title,body


In [ ]:
# app_list=app_list[0:1000]

In [ ]:
len(app_list)

2553

In [ ]:
for aidx, app in enumerate(app_list[2000:]):
    sys.stdout.write('Processing ... {} / {}\n'.format(aidx+1, len(app_list)))
    
    print(app[0])

    
    if aidx==0:
        continue
#     app_id = app['ID']

#     if 'ID_2' in header and app['ID_2'] !='missing':
#         app_id = app['ID_2']
#     if app_id == 'missing':
#         app_id = -1
#     else:
    app_id = int(app[0])
    curr.execute('insert into Apps (appID, appName, genre, url, baseid, baseapp) '\
                 'values (?,?,?,?,?,?)', (app[0], app[0], app[0], app[0],app[0], app[0]))
    conn.commit()
    # get reviews
    if app_id <=0:
        sys.stdout.write('No such app found on itunes. \n')
        continue
    
    url = "https://itunes.apple.com/us/customer-reviews/id%s?dataOnly=true&displayable-kind=11" % app_id

    for ttt in range(number_of_trying):
        try:
            response = requests.get(url, headers=headers)
            break
        except:
            response = None

    if not response:
        sys.stdout.write('Error when connecting to iTunes. \n')
        continue
    total_summary = response.json()
    all_number_of_reviews = total_summary.get("totalNumberOfReviews")
    if all_number_of_reviews<=0:
        sys.stdout.write('No reviews to retrieve. \n')
        continue
    sys.stdout.write('# Total reviews to retrieve for {}: {}\n'.format(app[0], all_number_of_reviews))
    
    all_reviews = []
    for startId in range(0, all_number_of_reviews, review_step):
        endId = startId + review_step
        if endId > all_number_of_reviews:
            endId = all_number_of_reviews
        url = "https://itunes.apple.com/WebObjects/MZStore.woa/wa/userReviewsRow?" \
          "id={}&dataOnly=true&displayable-kind=11&startIndex={}&endIndex={}&sort=0&appVersion=all".format(app_id, 
                                                                                                           startId, 
                                                                                                           endId)
        for ttt in range(number_of_trying):
            try:
                response = requests.get(url, headers=headers)
                break
            except:
                response = None
        if not response:
            sys.stdout.write('Error when connecting to iTunes. \n')
            break
        
        reviews = response.json()
        review_list = reviews.get('userReviewList')
        all_reviews.extend(review_list)
        sys.stdout.write('We now have: {}\n'.format(len(all_reviews)))
    for review in all_reviews:
        curr.execute('insert into Reviews (reviewID, appID, timestamp, rating, name, title, body) '\
                 'values (?,?,?,?,?,?,?)', (review.get('userReviewId'), app_id, 
                                            review.get('date'), review.get('rating'),
                                            review.get('name'), review.get('title'),
                                            review.get('body')))
    conn.commit()
    sys.stdout.write('{} reviews writen to DB...\n'.format(len(all_reviews)))
sys.stdout.write('Done.')

Streaming output truncated to the last 5000 lines.
We now have: 318000
We now have: 319000
We now have: 320000
We now have: 321000
We now have: 322000
We now have: 323000
We now have: 324000
We now have: 325000
We now have: 326000
We now have: 327000
We now have: 328000
We now have: 329000
We now have: 330000
We now have: 331000
We now have: 332000
We now have: 333000
We now have: 334000
We now have: 335000
We now have: 336000
We now have: 337000
We now have: 338000
We now have: 339000
We now have: 340000
We now have: 341000
We now have: 342000
We now have: 343000
We now have: 344000
We now have: 345000
We now have: 346000
We now have: 347000
We now have: 348000
We now have: 349000
We now have: 350000
We now have: 351000
We now have: 352000
We now have: 353000
We now have: 354000
We now have: 355000
We now have: 356000
We now have: 357000
We now have: 358000
We now have: 359000
We now have: 360000
We now have: 361000
We now have: 362000
We now have: 363000
We now have: 364000
We now ha

In [ ]:
len(all_reviews)

164

In [ ]:
reviews=pd.read_sql_query("SELECT * from Reviews",conn)

In [ ]:
apps=pd.read_sql_query("SELECT * from Apps",conn)

In [ ]:
reviews.shape

(2536637, 8)

In [ ]:
apps.shape

(552, 6)

In [ ]:
reviews.to_csv("SnowballLayer2Reviews3.csv")

In [ ]:
test=pd.read_csv("SnowballLayer2Reviews3.csv",index_col=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
test.head()
# print(test.shape)

,recordID,reviewID,appID,timestamp,rating,name,title,body
0,1.0,7.160449e+09,1.055315e+09,2021-03-30T00:37:40Z,1.0,Germ1382744138,App deleted everything!,Got the app and set it up in both mine and the...
1,2.0,7.159533e+09,1.055315e+09,2021-03-29T18:31:51Z,1.0,Midnight loves Dmacs channels,THIS APP GAVE ME A VIRUS,BRO THIS APP JUST BROKE EVERYTHING ALL MY APPS...
2,3.0,7.149008e+09,1.055315e+09,2021-03-27T02:46:27Z,3.0,fishfish12345,Useless,My kid disabled it the third day it was on his...
3,4.0,7.148343e+09,1.055315e+09,2021-03-26T22:31:42Z,1.0,this app is hproble,Do not get this app,This app is horrible.I am a teenage girl and m...
4,5.0,7.143913e+09,1.055315e+09,2021-03-25T18:30:22Z,2.0,85thKing,"Only partially worked, customer support lacking",Location tracking only worked intermittently. ...


In [ ]:
reviews[reviews['appID']==911410294]

,recordID,reviewID,appID,timestamp,rating,name,title,body
3703132,3703133,7137944669,911410294,2021-03-24T05:10:04Z,1,N.MK7,Terrible dating experience,Very poor profiles of guys on this app. No con...
3703133,3703134,7073687639,911410294,2021-03-06T23:54:48Z,1,scorpionz4,Not organized app,Need to manage location search. Every time I h...
3703134,3703135,7064711396,911410294,2021-03-04T16:27:20Z,5,Dfcg9835,buzz Arab,مرحبا نزلت صوره في الموقع ولحد الان لم تتم الم...
3703135,3703136,6926313921,911410294,2021-01-29T02:37:12Z,1,sunheart88,Fake Muslim men,This site is set up great but I do wish they d...
3703136,3703137,6907064642,911410294,2021-01-24T08:29:03Z,5,alifthinn,Super,Trusted application
...,...,...,...,...,...,...,...,...
3703275,3703276,1283426391,911410294,2015-11-07T22:09:38Z,5,Ameo_us,This app doesn't make any sense,After making my second account using a differe...
3703276,3703277,1277820975,911410294,2015-10-27T10:12:26Z,5,محمد الفجر الجديد,baghdad,great
3703277,3703278,1267678684,911410294,2015-10-06T07:18:40Z,4,Gefars,برنامج جيد,برنامج جيد لكن لم ينتشر في جميع الدول
3703278,3703279,1252985080,911410294,2015-09-04T13:22:46Z,1,Jack Numan,BuzzArab Great App,BuzzArab must do away with faceless members. N...


**Selecting Apps**

In [ ]:
import pandas as pd

In [ ]:
reviews3=pd.read_csv("SnowballLayer2Reviews3.csv",index_col=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
reviews3.shape

(2536673, 8)

In [ ]:
len(reviews3['appID'].unique())

485

In [ ]:
reviews2=pd.read_csv("SnowballLayer2Reviews2.csv",index_col=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
len(reviews2['appID'].unique())

1885

In [ ]:
fulldf=pd.concat([reviews2,reviews3],axis=0)

In [ ]:
seeddataset=pd.read_csv("/content/drive/My Drive/UIA/Rougue Score/RogueScoresTill1687Apps.csv")

In [ ]:
apps=fulldf['appID'].unique().tolist()

In [ ]:
lst=[]
for x in apps:
  if(x not in seeddataset['appID'].unique().tolist()):
    lst.append(x)

In [ ]:
len(lst)

2208

In [ ]:
newreviews=fulldf[fulldf['appID'].isin(lst)]

In [ ]:
filter=newreviews.groupby(['appID']).size()

In [ ]:
Lessthan10k=newreviews[newreviews['appID'].isin(filter[filter<=10000].index)]

In [ ]:
Lessthan10k.reset_index(drop=True,inplace=True)

In [ ]:
Lessthan10k.to_csv("Lessthan10kReviewsLayer2.csv")

In [ ]:
Morethan10k=newreviews[newreviews['appID'].isin(filter[filter>10000].index)]

In [ ]:
Morethan10k.reset_index(drop=True,inplace=True)

In [ ]:
Morethan10k.to_csv("Morethan10kReviewsLayer2.csv")

In [ ]:
Morethan10k.shape

(3403147, 8)